Benchmarking integrators from 3 langs. Matlab has a huge overhead

In [116]:
%%time
#### Python

import scipy.stats as spstat
import scipy.integrate as spint
f = spstat.gamma(a=1).cdf
for i in range(int(1E4)):
    b= spint.quad(f,0,1)
print b

(0.3678794411714423, 4.08428225874771e-15)
CPU times: user 22.9 s, sys: 8 ms, total: 22.9 s
Wall time: 22.9 s


In [117]:
%%time
### Julia
lines = ''''
using Distributions
using Cubature
D = Gamma(1)
f = x ->begin cdf(D,x);end
reps = 1E4
@time for j in 1:reps 
    b = quadgk(f,0,1)
    end
@time for j in 1:reps 
    b = hquadrature(f,0,1)
    end
b = quadgk(f,0,1)
println(b)' '''
!echo {lines} > test.jl
!julia test.jl

  0.917228 seconds (1.34 M allocations: 35.109 MiB, 1.27% gc time)
  0.306467 seconds (839.09 k allocations: 23.614 MiB, 2.00% gc time)
(0.3678794411714423, 5.551115123125783e-17)
CPU times: user 100 ms, sys: 20 ms, total: 120 ms
Wall time: 5.8 s


In [118]:
%%time
### Matlab
lines = '''"start = tic;
f = @(x)gamcdf(x,1,1);
for i = 1:1E4
    b = integral(f,0,1);
end
disp(b)
fprintf('%s elapsed: %f s\n', mfilename,toc(start))" ''' 
!echo {lines} >'test.m'
!matlab -nodisplay -r "run('./test.m');exit;"

=
                            < M A T L A B (R) >
                  Copyright 1984-2016 The MathWorks, Inc.
                   R2016b (9.1.0.441655) 64-bit (glnxa64)
                             September 7, 2016

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 
    0.3679

test elapsed: 5.759034 s>CPU times: user 240 ms, sys: 72 ms, total: 312 ms
Wall time: 13.5 s
